In [1]:
pip install keybert sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [keybert]m2/4 [rich]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install PyPDF2 python-docx

In [ ]:
pip install langchain_google_genai numpy scikit-learn pandas

In [ ]:
!pip install -q transformers accelerate langchain langchain-openai langchain-community  google-cloud-aiplatform

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## LIBRARIES


In [ ]:
import re
import pandas as pd
from collections import Counter
from nltk.tokenize import word_tokenize
import random
import numpy as np
from scipy.stats import entropy
from nltk.corpus import stopwords
import os
# from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
# import spacy
# from gensim import corpora, models
import json
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

## DOCUMENT Conversion

In [ ]:
doc_folder_path = '/content/drive/MyDrive/Grievance Scenario Docs'

In [ ]:
import docx
import os

def convert_docx_with_tables_to_text(docx_path):
    """
    Extracts text from a .docx file, correctly handling both paragraphs and tables.
    This is the corrected version that avoids the '.body' attribute error.

    Args:
        docx_path (str): The file path to the .docx file.

    Returns:
        str: The extracted and formatted text, or an empty string if an error occurs.
    """
    try:
        document = docx.Document(docx_path)
        full_text = []

        # First, extract text from all paragraphs
        for para in document.paragraphs:
            full_text.append(para.text)

        # Then, extract text from all tables
        for table in document.tables:
            # Add a marker to show where a table starts
            full_text.append("\n--- TABLE START ---")
            for row in table.rows:
                row_text = [cell.text.strip() for cell in row.cells]
                # Join cells with a separator for readability
                full_text.append("\t|\t".join(row_text))
            # Add a marker to show where a table ends
            full_text.append("--- TABLE END ---\n")

        return "\n".join(full_text)

    except Exception as e:
        print(f"Error reading DOCX file '{docx_path}': {e}")
        return ""

In [ ]:
def read_text_file(txt_file_path):
    # 1. Define the path to your text file
    full_content = "" # Initialize the variable to hold the text

    try:
        with open(txt_file_path, 'r', encoding='utf-8') as file:
            full_content = file.read()
        print("✅ File imported successfully into the variable.")

    except FileNotFoundError:
        print(f"❌ Error: The file was not found at '{txt_file_path}'")
    except Exception as e:
        print(f"❌ An error occurred: {e}")

    return full_content

In [ ]:
import os

# The path to your folder in Google Drive
doc_folder_path = '/content/drive/MyDrive/Grievance Scenario Docs'

try:
    # Get a list of all files and directories in the specified path
    all_files = os.listdir(doc_folder_path)

    print(f"Reading files from: {doc_folder_path}\n")

    # Loop through each file in the list
    for file_name in all_files:
            # Create the full path to the file
            full_path = os.path.join(doc_folder_path, file_name)

            # Check if the path is actually a file (and not a directory)
            # --- Add your file processing logic here ---
            # For example, you can call the text extraction functions
            # from the previous answer based on the file extension.
            #
            # if full_path.endswith('.pdf'):
            #     text = convert_pdf_to_text(full_path)
            #     print(text[:100]) # Print first 100 characters
            # elif full_path.endswith('.docx'):
            #     text = convert_docx_to_text(full_path)
            #     print(text[:100]) # Print first 100 characters
            # --------------------------------------------
            extracted_text = convert_docx_with_tables_to_text(full_path)

            # check if extracted file exists or not
            if extracted_text:
                output_txt_file = f"{file_name.replace('.docx','.txt')}"
                if os.path.isfile(full_path)    :
                    print(f"Processing file: {full_path}")
            # Save the extracted text to a .txt file
                with open(output_txt_file, 'w', encoding='utf-8') as f:
                    f.write(extracted_text)
            else:
                print(f"❌ Error: The file was not found at '{full_path}'")
                print("Please make sure your Google Drive is mounted and the file name is correct.")

except FileNotFoundError:
    print(f"Error: The folder path was not found.")
    print("Please make sure your Google Drive is mounted and the path is correct.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
from keybert import KeyBERT

def keyword_extract(txt_file_path):

    # Using the same doc_text from the previous example
    doc_text = read_text_file(txt_file_path)

    kw_model = KeyBERT(model='all-MiniLM-L6-v2')

    # Extract more advanced keywords and phrases
    advanced_keywords = kw_model.extract_keywords(
        doc_text,
        keyphrase_ngram_range=(1, 3),  # Consider phrases of 1, 2, or 3 words
        stop_words='english',
        use_mmr=True,                  # Use Maximal Marginal Relevance for diversity
        diversity=0.7,                 # Set the diversity score (0 to 1)
        top_n=1000                       # Get the top 10 diverse keywords/phrases
    )

    print("\n--- Advanced & Diverse Keywords ---")
    print(type(advanced_keywords))
    print(advanced_keywords)
    return advanced_keywords

In [ ]:
kw = keyword_extract('/content/DRC Document (2) (1).txt')

## CODE

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

# ✅ Initialize Relevance Checker LLM
model = ChatOpenAI(
    model="/mnt/nvme-storage/nitin/.cache/huggingface/models--openai--gpt-oss-120b/snapshots/b5c939de8f754692c1647ca79fbf85e8c1e70f8a",
    openai_api_base="http://183.82.7.228:9532/v1/",
    openai_api_key="DUMMY_KEY",
    temperature=0.5,
)

/tmp/ipython-input-545120585.py:7: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(


In [ ]:
# @title keyword_extract
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import json
import re

def extract_keywords_with_chatopenai(text_content: str) -> list:
    """
    Uses a self-hosted ChatOpenAI model via LangChain to extract keywords.

    Args:
        text_content: The document text to analyze.

    Returns:
        A list of keywords or an empty list if an error occurs.
    """
    # 1. Initialize the ChatOpenAI model with your provided configuration
    try:
        model = ChatOpenAI(
            model="/mnt/nvme-storage/nitin/.cache/huggingface/models--openai--gpt-oss-120b/snapshots/b5c939de8f754692c1647ca79fbf85e8c1e70f8a",
            openai_api_base="http://183.82.7.228:9532/v1/",
            openai_api_key="DUMMY_KEY",
            temperature=0.5,
        )
    except Exception as e:
        print(f"Error initializing the model: {e}")
        return []

    # 2. The prompt remains the same—it's model-agnostic
    prompt = f"""
    You are an expert in analyzing financial and technical documents related to UPI payments.
    Your task is to identify and extract the most important and relevant keywords and keyphrases
    from the following document text.

    Instructions:
    1. Identify the top 1000 keywords and keyphrases related to the Document Text provided below. Note that the keywords
       be strictly relevant to the domain of UPI.
    2. Do not mix 2 different words like 'analyticsengine','reportingtool' etc.
    3. Keyphrases should be between 1 and 2 words long (strict).
    4. Focus on terms that represent the core concepts, actions, and entities.
    5. Do not include generic terms (very strict).
    6. While returning order the words in descending order according to their relation to the overall document.
    7. Return the result strictly as a single JSON object with one key: "keywords".
       The value should be a list of the extracted strings.

    Document Text:
    ---
    {text_content}
    ---

    JSON Output:
    """

    # 3. Create the message payload and invoke the model
    messages = [HumanMessage(content=prompt)]

    try:
        response = model.invoke(messages)
        # LangChain's response object has a 'content' attribute for the text
        response_text = response.content

        # 4. Clean and parse the JSON response (same as before)
        cleaned_response = re.sub(r"```json\s*|\s*```", "", response_text.strip())
        result = json.loads(cleaned_response)

        return result.get("keywords", [])

    except Exception as e:
        print(f"An error occurred during API call or parsing: {e}")
        # 'response_text' might not exist if the API call itself failed
        raw_response = locals().get('response_text', 'No response received from model.')
        print(f"Raw model response was: {raw_response}")
        return []



In [ ]:
# doc_text = read_text_file('/content/DRC Document (2) (1).txt')
# kw = extract_keywords_with_chatopenai(doc_text)

In [ ]:
# txt_files = [f for f in os.listdir() if f.endswith('.txt')]

In [ ]:
# txt_files_2 = ['UPI-Lite Failure Scenarios_.txt',
#  'M16 Scenario (1).txt',
#  'Frauds and chargeback scenarios.txt']

In [ ]:
# kw = []
# for path in txt_files:
#     doc_text = read_text_file(path)
#     kw.extend(extract_keywords_with_chatopenai(doc_text))


In [ ]:
# @title clean_text()
def clean_text(text):
    """ download nltk stopwords """
    try:
        stopwords.words('english')
    except LookupError:
        print("Downloading NLTK stopwords...")
        nltk.download('stopwords')
        print("Download complete.")

    """Cleans text for statistical analysis (TF-IDF, LDA)."""
    text = text.lower()
    # text = translator.translate(text, dest="en")
    text = re.sub(r'[^a-z\s]', '', text) # Remove non-alphabetic characters
    stop_words = set(stopwords.words('english'))
    words = text.split()
    # Also remove single-letter words
    filtered_words = [word for word in words if word not in stop_words and len(word) > 2]
    return filtered_words

In [ ]:
# @title set_generator
def segment(response_words, similarity_matrix, SIMILARITY_THRESHOLD):
    in_domain_words = set()
    out_of_domain_words = set()

    # Check if similarity_matrix is empty
    if similarity_matrix.size == 0:
        return in_domain_words, out_of_domain_words


    for i, word in enumerate(response_words):
        # Find the highest similarity score for the current word against all keywords.
        max_similarity = similarity_matrix[i].max()

        if max_similarity >= SIMILARITY_THRESHOLD:
            in_domain_words.add(word)
        else:
            out_of_domain_words.add(word)
    return in_domain_words, out_of_domain_words


def set_generator(domain_keywords, user_response, assistant_response, model, keyword_embeddings):

    # --- Step 1 & 2: Define Keywords and Load Model ---

    # These are the keywords/phrases that define your UPI domain.
    # In a real application, you would load these from your extraction process.
    domain_keywords = kw

    # Load a pre-trained model optimized for semantic similarity.
    # print("Loading sentence transformer model...")
    # model = SentenceTransformer('all-MiniLM-L6-v2')


    # --- Step 3: Generate Keyword Embeddings ---

    # print("Generating embeddings for domain keywords...")
    # This creates a vector for each keyword, forming your "Domain Knowledge Base".
    # keyword_embeddings = model.encode(domain_keywords)


    # --- Step 4 & 5: Process a Conversation and Calculate Similarity ---

    # An example response from an assistant to be checked.
    # assistant_response = """"Your transaction was declined due to a **Bank Server Timeout**. This usually happens when your bank’s server is temporarily unavailable or overloaded, causing the transaction to fail even though the amount was deducted.

    # Here’s what you can do next:

    # 1. **Wait for a refund**: Mst banks automatically rifund failed txns within **1–3 business days**.
    # 2. **Check your bank statement**: Confirm if the amount has been credited back.
    # 3. **Raise a dispute** (if needed): If the refund doesn’t come through, contact your bank to raise a dispute.

    # Would you like help with filing a complaint with your bank?"""

    assistant_response = clean_text(assistant_response)
    assistant_response = ' '.join(assistant_response)

    user_response = clean_text(user_response)
    user_response = ' '.join(user_response)

    # Split the response into individual words for analysis.
    asst_response_words = assistant_response.lower().replace('.', '').split()
    user_response_words = user_response.lower().replace('.', '').split()

    # print(f"\nAnalyzing response: '{assistant_response}'")

    # Generate embeddings for each word in the response.
    asst_response_word_embeddings = model.encode(asst_response_words)
    user_response_word_embeddings = model.encode(user_response_words)

    # print(asst_response_word_embeddings)
    # print(user_response_word_embeddings)
    # Calculate the cosine similarity between each response word and all domain keywords.
    # This creates a matrix where each row is a response word and each column is a domain keyword.
    # Check if embedding arrays are empty before calculating similarity
    if user_response_word_embeddings.size > 0 and keyword_embeddings.size > 0:
        user_similarity_matrix = cosine_similarity(user_response_word_embeddings, keyword_embeddings)
    else:
        user_similarity_matrix = np.array([]) # Use an empty numpy array if no embeddings

    if asst_response_word_embeddings.size > 0 and keyword_embeddings.size > 0:
        asst_similarity_matrix = cosine_similarity(asst_response_word_embeddings, keyword_embeddings)
    else:
        asst_similarity_matrix = np.array([]) # Use an empty numpy array if no embeddings


    # Set your confidence threshold. You may need to tune this value (0.5 to 0.7 is a good start).
    SIMILARITY_THRESHOLD = 0.5

    # --- Step 6: Classify Words ---

    asst_in_domain_words, asst_out_of_domain_words = segment(asst_response_words, asst_similarity_matrix, SIMILARITY_THRESHOLD)
    user_in_domain_words, user_out_of_domain_words = segment(user_response_words, user_similarity_matrix, SIMILARITY_THRESHOLD)

    # for i, word in enumerate(response_words):
    #     # Find the highest similarity score for the current word against all keywords.
    #     max_similarity = similarity_matrix[i].max()

    #     if max_similarity >= SIMILARITY_THRESHOLD:
    #         in_domain_words.add(word)
    #     else:
    #         out_of_domain_words.add(word)

    # --- Results ---

    # print("\n--- Domain Analysis Results ---")
    # print(f"Similarity Threshold: {SIMILARITY_THRESHOLD}")
    # print("\n✅ Words WITHIN UPI Domain:")
    # print(sorted(list(in_domain_words)))

    # print("\n❌ Words OUTSIDE UPI Domain:")
    # print(sorted(list(out_of_domain_words)))

    return asst_in_domain_words, asst_out_of_domain_words, user_in_domain_words, user_out_of_domain_words

In [ ]:
# @title translation
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

def translate_to_proper_english(raw_text: str, llm_model: ChatOpenAI) -> str:
    """
    Translates a user's message containing typos, shortcuts, and Hinglish
    into clean, standard English using an LLM.

    Args:
        raw_text: The user's original message string.
        llm_model: An instantiated LangChain ChatOpenAI model.

    Returns:
        The cleaned and translated English string.
    """
    # This detailed prompt is the core of the solution.
    prompt = f"""
    You are a language expert specializing in cleaning up user support messages for analysis.
    Your task is to translate the following text into clear and grammatically correct English.

    The original text may contain a mix of the following:
    1.  Spelling mistakes and typos (e.g., "rifund", "faild").
    2.  Common chat shortcuts and abbreviations (e.g., "plz", "u", "txns", "amt").
    3.  A mix of Hindi and English, also known as Hinglish (e.g., "paisa wapas kab aayega").

    CRITICAL INSTRUCTION: You must preserve the core meaning and all key entities from the
    original text. This includes amounts, transaction IDs, dates, names, or any other specific data.
    Do not add any new information or commentary.

    Your final output should ONLY be the corrected English text and nothing else.

    Original Text: "{raw_text}"

    Cleaned English Text:
    """

    try:
        messages = [HumanMessage(content=prompt)]
        response = llm_model.invoke(messages)
        # The .content attribute holds the string from the model's response
        return response.content.strip()
    except Exception as e:
        print(f"An error occurred during LLM translation: {e}")
        # If the LLM fails, return the original text to avoid breaking the workflow
        return raw_text

# --- Main Execution Block ---

# 1. Initialize the ChatOpenAI model with your configuration
# Using a low temperature (e.g., 0.1) is best for factual tasks like translation.
# try:
#     llm_translator = ChatOpenAI(
#         model="/mnt/nvme-storage/nitin/.cache/huggingface/models--openai--gpt-oss-120b/snapshots/b5c939de8f754692c1647ca79fbf85e8c1e70f8a",
#         openai_api_base="http://183.82.7.228:9532/v1/",
#         openai_api_key="DUMMY_KEY",
#         temperature=0.1,
#     )
# except Exception as e:
#     print(f"Failed to initialize the LLM. Please check your configuration. Error: {e}")
#     exit()

# # 2. List of example user messages to test
# test_messages = [
#     "my txn of 500rs faild, plz hlp me",
#     "paisa wapas kab aayega? transaction id is T123456789",
#     "mujhe abhi tak rifund nahi mila for my last payment",
#     "mera upi payment got stk... wat to do?",
#     "i did a recharge of 99 but it did not reflect, pls check",
#     "yrr ye bank server hamesha down rehta hai",
# ]

# print("--- Translating User Messages into Proper English ---\n")

# # 3. Process each message and display the results
# for message in test_messages:
#     translated_message = translate_to_proper_english(message, llm_translator)
#     print(f"Original  : {message}")
#     print(f"Translated: {translated_message}")
#     print("-" * 20)

In [ ]:
# kwords = list(set(kw))
# kwords = (' ').join(kw)
# kwords = kwords.lower()
# with open('/content/keywords.txt', 'w') as f:
#     f.write(kwords)

In [ ]:
# @title system_prompt
system_prompt = """
You are an AI payment support agent trained to handle **UPI transaction grievances ONLY**, but you may also answer **general UPI-related questions** if the user asks.

Your job is to guide users step-by-step with empathy, clarity, and accuracy. Never assume. You must strictly follow the assistant flow and only simulate responses based on provided details. Do not include unnecessary information unless the user explicitly asks.

---

ASSISTANT FLOW

1. Start with Empathy & Collect Details
- Greet the user warmly and show empathy
- Ask for these 3 details in this transaction amount,date and time resolove if user answers yeaterday,today,afternoon,evening,night,now


2. Simulate Transaction Lookup
- If all 3 details are present, proceed with simulation
- Show message: *"Thanks! I’m checking for a matching transaction using the information you provided…"*
- Simulate **realistic but fake** transaction details:
  • Amount: ₹1999
  • Time: 3:05 PM on 21/07/2025
  • Payee: Coursera Inc
  • Status: Failed / Pending / Successful / etc.
- Do **not** use or reference real data

3.If No Match Found
- Say: *"I couldn’t find a transaction that matches those details."*
- Ask the user to recheck or upload a screenshot
- Return to Step 1 if needed

4. If Match Found, Confirm
- Summarize the transaction:
  • Amount, Date, Time, Payee
- Ask: *"Is this the one you were referring to?"*

5. Explain the Issue (If Confirmed)
- Respond based on **transaction status** and known reason

  • **Failed**: Use one of the following fake but realistic reasons:
    - **Duplicate Mandate Request**
    - **Incorrect UPI Handle Format**
    - **Insufficient Balance**
    - **Bank Server Timeout**
    - **Mandate Token Expired**
    - **MDR Rejected by Acquirer**
    - **VPA Resolution Failed**

    Example:
    > "Your transaction was declined due to a **Duplicate Mandate Request**. This usually happens when the same payment is attempted more than once and your bank blocks the duplicate to prevent double charging."

  • **Pending**:
    > "This transaction is currently marked as **Pending**. Most pending UPI transactions settle automatically within **T+2 working days**."

  • **Successful but confused**:
    > "Your course access should be fine. The original payment went through, and this second attempt was declined as a duplicate."

  • **Disputed or Fraud**: move to next steps below

- **DO NOT** mention general reasons (like PIN error, network, low balance) unless they match the failure cause or the user explicitly asks.

6. Handle Fraud or Collect Request Scams
- If it’s a fraud complaint:
  1. Ask user to report to their bank immediately
  2. Instruct them to file a complaint at **https://www.cybercrime.gov.in** or call **1930**
- Offer to help **draft the complaint** if needed

7. Guide on Filing Complaint (If User Agrees)
- Provide clear steps to:
  • File complaint on bank's dispute portal
  • File complaint on Cybercrime website
- Mention supporting documents: screenshots, SMS, bank statement

8. Explain What Happens After Filing
- Complaints go into review
- Timeline: **up to 90 days** for resolution
- Encourage them to follow up with their bank if needed

9.If Still No Match or Confusion
- Ask for details again or request screenshot
- Never simulate without complete 3-point info

10.Always End With Disclaimer
**"Note: I’m an AI assistant and can guide you, but you need to file the actual complaint through your bank or the official portals."**

---

💬 GENERAL UPI QUESTIONS (Optional)

You may answer general questions such as:
- How UPI works
- What is the UPI limit
- What is UPI Autopay
- Common UPI errors (e.g., U16, U07)
- How to reset UPI PIN or switch bank account

Answer style:
- Clear and concise
- Use 3–4 bullet points max
- Friendly tone with emojis

---

RESPONSE STYLE & RULES
- Always respond in English even if the user query is in Hindi or any other language
- Empathetic, helpful, professional tone
- Use **bold** for key details (e.g., **Amount**, **Status**)
- Use emojis where natural
-  Mobile-friendly format: 1–2 short paragraphs or bullet points
- Never use ALL CAPS — write “Unknown”, not “UNKNOWN”

---

DO NOT ADD EXTRA INFORMATION

-  Do NOT give generic reasons like “Insufficient funds” or “Incorrect PIN” unless the user asks
-  If the issue is already known (e.g., **Duplicate Mandate**), explain **only that**
- Do NOT suggest retrying payment, contacting support, or checking balance **unless asked**
- Keep answers relevant, brief, and strictly aligned with what the user asked
- Only guide forward when the user confirms or requests it

"""


In [ ]:
# @title load_assistant
def load_assistant_model():
#later it can be modified so that different models are allowed to be called
    mistral_assistant = ChatOpenAI(
        model="NPCI_Greviance",
        openai_api_base="http://183.82.7.228:9519/v1",
        openai_api_key="sk-fake",
        temperature=0.2,
    )
    return mistral_assistant

In [ ]:
# @title actual conv generation
import json
import re
import time
from typing import Dict, List, Optional, Literal

from langchain.chat_models import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# =========================
# Config
# =========================
LAST_K_TURNS = 12  # how many recent turns of conversation history to include


# =========================
# Helpers
# =========================
SMART_QUOTES = {
    "“": '"', "”": '"', "„": '"', "‟": '"',
    "’": "'", "‘": "'", "‚": "'", "‛": "'",
}

def _normalize_quotes(text: str) -> str:
    for k, v in SMART_QUOTES.items():
        text = text.replace(k, v)
    return text

def _strip_code_fences(text: str) -> str:
    # Remove ```json ... ``` or ``` ... ```
    text = re.sub(r"```json\s*([\s\S]*?)```", r"\1", text, flags=re.IGNORECASE)
    text = re.sub(r"```\s*([\s\S]*?)```", r"\1", text)
    return text.strip()

# =========================
# Main evaluation
# =========================

def run_multi_dimensional_evaluation(
    user_questions: List[str],
    expected_answers: List[str],
    user_goal: str,
    system_prompt: str,
    *,
    mode: Literal["assistant", "expected", "both"] = "assistant",
    assistant_answers: Optional[List[str]] = None,
    eval_temperature: float = 0.0,
) -> List[dict]:
    """
    Evaluate answers (assistant/expected) across multiple scoring dimensions using GPT-OSS-120B.
    """

    # Deterministic judge; use stop tokens to discourage trailing prose
    # evaluator_llm = ChatOpenAI(
    #     model="/mnt/nvme-storage/nitin/.cache/huggingface/models--openai--gpt-oss-120b/snapshots/b5c939de8f754692c1647ca79fbf85e8c1e70f8a",
    #     openai_api_base="http://183.82.7.228:9532/v1/",
    #     openai_api_key="DUMMY_KEY",
    #     temperature=eval_temperature,  # keep 0 for consistency
    #     max_tokens=1500,                 # judges need very few tokens
    #     max_retries=3,

    # )
    messages: List = [SystemMessage(content=system_prompt)]

    ASSISTANT_POLICY = """
- Scope: UPI grievances/FAQs only (failed/pending/declined payments, UPI Lite, Autopay/mandates, fraud handling, complaint/dispute paths).
- No speculation: Only state causes when confirmed; request minimal details first.
- Allowed data: amount, date, time, optional screenshot (with redact reminder).
- Forbidden: UPI PIN, OTP, passwords, CVV, Aadhaar, PAN, full account numbers.
- Capabilities: Can simulate lookups and explain tool outputs.
- Brand mentions only if explicitly established.
"""
    # =========================
    # Main loop
    # =========================
    conversation_turns: List[dict] = []

    # load mistral assistant
    mistral_assistant = load_assistant_model()

    for idx, question in enumerate(user_questions):
        # Fresh message scaffold per item to prevent history bleed
        #messages: List = [SystemMessage(content=system_prompt)]
        expected_reply = expected_answers[idx] if idx < len(expected_answers) else ""
        assistant_reply = None

        # Choose mode for generating the candidate answer
        if mode in ("assistant", "both"):
            if assistant_answers is not None:
                assistant_reply = assistant_answers[idx]
            else:
                messages.append(HumanMessage(content=question))
                ai = mistral_assistant.invoke(messages)  # <-- use your assistant, NOT the judge
                assistant_reply = (ai.content or "").strip()
                messages.append(AIMessage(content=assistant_reply))

        if mode == "expected":
            messages.append(HumanMessage(content=question))
            messages.append(AIMessage(content=expected_reply))

        # Build conversation json for context
        convo_json = []
        for msg in messages:
            if isinstance(msg, SystemMessage):
                continue
            role = "user" if isinstance(msg, HumanMessage) else "assistant"
            convo_json.append({"role": role, "content": msg.content.strip()})

        # Record evaluation
        record: Dict[str, any] = {"user": question}
        if mode == "assistant":
            record["assistant"] = assistant_reply
            # record["assistant_eval"] = _evaluate_reply(question, assistant_reply, convo_json)
        elif mode == "expected":
            record["assistant"] = expected_reply
            # record["expected_eval"] = _evaluate_reply(question, expected_reply, convo_json)
        else:  # both
            record["assistant"] = assistant_reply
            record["expected"] = expected_reply
            # record["assistant_eval"] = _evaluate_reply(question, assistant_reply, convo_json)
            # record["expected_eval"] = _evaluate_reply(question, expected_reply, convo_json)

        conversation_turns.append(record)

    return conversation_turns

In [ ]:
# @title parse_user_assistant_pairs
def parse_user_assistant_pairs(convo):
    user_questions = []
    expected_answers = []

    i = 0
    while i < len(convo):
        curr_msg = convo[i]

        if curr_msg.get("role", "").lower() == "user":
            user_question = curr_msg.get("content", "")
            i += 1
            assistant_reply_parts = []

            # Collect all assistant responses until next user or end
            while i < len(convo) and convo[i].get("role", "").lower() != "user":
                role = convo[i].get("role", "").lower()
                content = convo[i].get("content")

                if role == "assistant" and isinstance(content, str):
                    assistant_reply_parts.append(content.strip())
                i += 1

            full_assistant_reply = "\n".join(assistant_reply_parts).strip()

            if full_assistant_reply:
                user_questions.append(user_question)
                expected_answers.append(full_assistant_reply)

        else:
            i += 1

    return user_questions, expected_answers


In [ ]:
# @title single_conv
def evaluate_single_conversation(convo_data, system_prompt, mode):
    try:
        user_qs, expected_as = parse_user_assistant_pairs(convo_data.get("messages", []))
        if not user_qs:
            # update_progress()
            return []

        results = run_multi_dimensional_evaluation(
            user_qs,
            expected_as,
            user_goal="upi transaction complaints",
            system_prompt=system_prompt,
            mode=mode
        )

        for res in results:
            res["conversation_id"] = convo_data.get("conversation_id", "unknown")

    except Exception as e:
        print(f"❌ Error while processing conversation: {e}")
        results = []

    time.sleep(0.5)
    # update_progress()
    print(results)
    return results


In [ ]:
# @title get_scores
def get_scores(conv_id, df_conv, upi_dom, model, keyword_embeddings):
    # df_json.iloc[4]['messages'][0]['content']
    # for i in range(df_json.shape[0]):
    #     # for j in range(len(df_json.iloc[i]['messages'])):
    #     for j in range(len(df_json.iloc[4]['messages'])):
    #         for key, value in df_json.iloc[i]['messages'][j].items():
    #             if (key == 'role') & (value == 'user'):
    #                 continue
    #             print(f"{key}: {value}")
    # set_upi =set(upi_dom)
    # print('\n printing upi_dom list', upi_dom)
    domain_scores = []
    relevance_scores = []
    redundancy_scores = []
    coherence_scores = []
    i=0

    user_id_set = set()
    asst_id_set = set()
    user_od_set = set()
    asst_od_set = set()

    # print("Loading sentence transformer model...")
    # model = SentenceTransformer('all-MiniLM-L6-v2')
    # print("Generating embeddings for domain keywords...")
    # # This creates a vector for each keyword, forming your "Domain Knowledge Base".
    # keyword_embeddings = model.encode(upi_dom)

    for index, row in df_conv.iterrows():
        set_asst_id, set_asst_od, set_user_id, set_user_od = set_generator(upi_dom, row['user_translation'], row['assistant'], model, keyword_embeddings)
        user_id_set = user_id_set.union(set_user_id)
        asst_id_set = asst_id_set.union(set_asst_id)
        user_od_set = user_od_set.union(set_user_od)
        asst_od_set = asst_od_set.union(set_asst_od)

    set_asst = asst_id_set.union(asst_od_set)
    set_user = user_id_set.union(user_od_set)
    set_upi = set(upi_dom)
    print('\n conv_id:',conv_id)
    print('\n assistant set: ',set_asst)
    print('\n user set: ',set_user)
    # print('\n upi',set_upi)

    p1 = user_id_set.intersection(asst_id_set) #exact domain
    p2 = user_id_set - p1
    p3 = set_asst.intersection(set_user) - p1
    p4 = asst_id_set - p1  ## looks problematic

    print('\n p1 set: ',p1)
    print('\n p2 set: ',p2)
    print('\n p3 set: ',p3)
    print('\n p4 set: ',p4)
    # print({'p1':p1},{'p2':p2},{'p3':p3},{'p4':p4},{'p5':p5},{'p7':p7})

    domain_score = len(p1)/len(set_asst)
    domain_scores.append(domain_score)

    relevance_score = (len(p1)+len(p3))/len(set_asst)
    relevance_scores.append(relevance_score)

    # redundancy
    redundancy_score = len(p4)/len(set_asst)
    redundancy_scores.append(redundancy_score)

    # coherence
    # coherence_score = 0.4*domain_score + 0.5*relevance_score - 0.1*redundancy_score
    # coherence_scores.append(coherence_score)

    df_result=pd.DataFrame()
    df_result['conv_id'] = [conv_id]
    df_result['domain_score'] = domain_scores
    df_result['relevance_score'] = relevance_scores
    df_result['redundancy_score'] = redundancy_scores
    # df_result['coherence_score'] = coherence_scores

    # print(domain_scores)
    # print(relevance_scores)
    # print(redundancy_scores)
    # print(coherence_scores)
    return df_result

In [ ]:
# @title generate_conversation_
import json
import pandas as pd
import time


total_tasks = 0

# def update_progress():
#     global progress_counter += 1
#     print(f"✅ Progress: {progress_counter}", flush=True)

def generate_conversation_from_jsonl_file(file_path, system_prompt, mode):
    final_results = []
    prog_counter = 0
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
        global total_tasks
        total_tasks = len(lines)

    print(f"📂 Total conversations: {total_tasks}")

    for idx, line in enumerate(lines):
        try:
            convo_data = json.loads(line.strip())
            result = evaluate_single_conversation(convo_data, system_prompt, mode)
            if result:
                final_results.extend(result)
            prog_counter += 1
            print(f"✅ Progress: {prog_counter}", flush=True)
        except json.JSONDecodeError as e:
            print(f"❌ JSON error at line {idx + 1}: {e}")

    df = pd.DataFrame(final_results)

    try:
        llm_translator = ChatOpenAI(
            model="/mnt/nvme-storage/nitin/.cache/huggingface/models--openai--gpt-oss-120b/snapshots/b5c939de8f754692c1647ca79fbf85e8c1e70f8a",
            openai_api_base="http://183.82.7.228:9532/v1/",
            openai_api_key="DUMMY_KEY",
            temperature=0.1,
        )
    except Exception as e:
        print(f"Failed to initialize the LLM. Please check your configuration. Error: {e}")

    user_translations = []
    for index, row in df.iterrows():
        user_translations.append(translate_to_proper_english(row['user'], llm_translator))
    # df = score_extract(df)
    df['user_translation'] = pd.Series(user_translations)

    # assistant_translations = []
    # for index, row in df.iterrows():
    #     assistant_translations.append(translate_to_proper_english(row['assistant']), llm_translator)
    # output_filename = f"{file_path.replace('.jsonl','')}_user_mistral.json"
    # save_conversation_data(final_results, output_filename)
    # print(f"\n✅ Saved evaluation results to {file_path.replace('.jsonl','')}.csv")
    return df


In [ ]:
# @title save_conversation_data()
def save_conversation_data(conversation_data, output_filename):
    try:
            # 'w' means open the file in write mode
            # encoding='utf-8' is a good practice for handling text
            with open(output_filename, 'w', encoding='utf-8') as json_file:
                # json.dump() writes the Python object to the file
                # indent=4 makes the JSON file nicely formatted and readable
                json.dump(conversation_data, json_file, indent=4)

            print(f"🎉 Successfully saved conversation data to '{output_filename}'")

    except Exception as e:
            print(f"❌ Error saving to JSON file: {e}")

In [ ]:
# @title convert_json_to_jsonl()
def convert_json_to_jsonl(json_file_path, output_jsonl):
    with open(json_file_path, 'r') as f:
        data = json.load(f)  # Load the entire JSON array

    with open(output_jsonl, 'w') as f_out:
        for item in data:
            json.dump(item, f_out)  # Write each object as a JSON string
            f_out.write('\n')      # Add a newline character


# file_path = '/content/coh_english_v6.json'
# output_jsonl = (file_path.replace(".json", ".jsonl"))
# convert_json_to_jsonl(file_path, output_jsonl)

In [ ]:
# @title JSON to JSONL call code
# CREATING JSONL FILES FOR EVALUATION

directory_path = "/content/drive/MyDrive/Hand_wrriten_json/evaluation new september/Coherence"
json_files = [f for f in os.listdir(directory_path) if f.endswith('.json')]
# print(json_files)
jsonl_files = []

# using ps.path.isdir() inside if else to create directory
if not os.path.isdir(os.path.join('/content', 'jsonl')):
    output_path = os.mkdir(os.path.join('/content', 'jsonl'))
    print('jsonl directory created!')
else:
    output_path = os.path.join('/content', 'jsonl')

print(json_files)
total_tasks = len(json_files)

# creating jsonl files
for i in range(total_tasks):
    json_file_path = os.path.join(directory_path, json_files[i])
    output_jsonl = (os.path.join(output_path, json_files[i])).replace(".json", ".jsonl")
    convert_json_to_jsonl(json_file_path, output_jsonl)
    jsonl_files.append(output_jsonl)


['coh_hinglish_v6.json', 'coh_english_v6.json', 'coh_broken_eng.json', 'coh_short_eng.json']


In [ ]:
df_concat = pd.DataFrame()
mode = 'assistant'
for i in range(0,4):
    df = generate_conversation_from_jsonl_file(jsonl_files[i], system_prompt, mode)
    df_concat = pd.concat([df_concat, df], axis=0)

df_concat.to_csv('/content/drive/MyDrive/upi_llm_content/generated_conversations.csv', index=False)

📂 Total conversations: 26
[{'user': 'baar baar payment fail ho raha hai ek hi shop pe. likh ke aa raha hai payee psp timeout. kya problem hai?', 'assistant': 'I\'m sorry to hear that you\'re facing issues with your UPI payments. The error message **"Payee PSP Timeout"** typically means that the recipient\'s payment service provider (PSP) is taking too long to respond or is temporarily unavailable.\n\nHere are a few possible reasons and solutions:\n\n- **Server Overload**: The recipient\'s bank or payment app might be experiencing high traffic.\n  - *Try again later.*\n\n- **Technical Issues**: There might be a temporary glitch on the recipient\'s side.\n  - *Contact the shop or merchant to check if others are facing the same issue.*\n\n- **Network Problems**: Slow or unstable internet on either your or the recipient\'s end can cause timeouts.\n  - *Ensure you have a stable internet connection and try again.*\n\nIf the issue persists, you may want to:\n- Try a different UPI app or bank 

In [ ]:
upi_dom = read_text_file('/content/drive/MyDrive/upi_llm_content/keywords.txt')
kw = upi_dom.split()
kw = list(set(kw))

✅ File imported successfully into the variable.


In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/upi_llm_content/generated_conversations.csv')
grp = list(df2.groupby('conversation_id'))
ids = df2['conversation_id'].unique()

In [ ]:
df2

In [ ]:
df_scores = pd.DataFrame()

print("Loading sentence transformer model...")
model = SentenceTransformer('all-MiniLM-L6-v2')

print("Generating embeddings for domain keywords...")
# This creates a vector for each keyword, forming your "Domain Knowledge Base".
keyword_embeddings = model.encode(kw)

for i in range(len(ids)):
    df_temp = get_scores(ids[i], grp[i][1], kw, model, keyword_embeddings)
    df_scores = pd.concat([df_scores, df_temp], axis=0)
del df_temp

df_scores['domain_norm'] = df_scores['domain_score']/df_scores['domain_score'].max()
df_scores['relevance_norm'] = df_scores['relevance_score']/df_scores['relevance_score'].max()
df_scores['redundancy_norm'] = (df_scores['redundancy_score'] - df_scores['redundancy_score'].min())/(df_scores['redundancy_score'].max() - df_scores['redundancy_score'].min())

df_scores['coherence'] = 0.4*df_scores['domain_norm'] + 0.5*df_scores['relevance_norm'] - 0.1*df_scores['redundancy_norm']

df_scores.reset_index(drop=True, inplace=True)
df_scores


In [ ]:
df_scores.to_csv('mistral_scores.csv',index=False)

In [ ]:
def create_radar(df_scores):
    import plotly.express as px

    df = pd.DataFrame()

    df['category'] = ['domain_norm', 'relevance_norm', 'redundancy_norm', 'coherence']
    df['scores'] = [df_scores['domain_norm'].mean(), df_scores['relevance_norm'].mean(), df_scores['redundancy_norm'].mean(), df_scores['coherence'].mean()]


    fig = px.line_polar(df, r='scores', theta='category', line_close=True,
                        title='Score Radar')
    fig.show()


In [ ]:
create_radar(df_scores)

In [ ]:
df_scores['domain_norm'].mean()
